# 11 RDD转DataFrame、建立Spark SQL tempTable

In [1]:
sc.master # 用的单机local模式

'local[*]'

In [2]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [3]:
global Path
if sc.master[0:5]=="local":
    Path="file:///home/s18221221/jupyter-workplace/data/" # 最终需要以/结尾
else:
    Path="hdfs://ubuntu62:9002/data/"
# 如果要在cluster模式运行（hadoop yarn 或Spark Stand alone） 请把文件上传到HDFS目录

# 建立RDD

In [4]:
RawUserRDD = sc.textFile(Path + "u.user")

In [5]:
RawUserRDD.count()

943

In [6]:
RawUserRDD.take(5)

['1|24|M|technician|85711',
 '2|53|F|other|94043',
 '3|23|M|writer|32067',
 '4|24|M|technician|43537',
 '5|33|F|other|15213']

In [7]:
userRDD = RawUserRDD.map(lambda line : line.split("|"))
userRDD.take(5)

[['1', '24', 'M', 'technician', '85711'],
 ['2', '53', 'F', 'other', '94043'],
 ['3', '23', 'M', 'writer', '32067'],
 ['4', '24', 'M', 'technician', '43537'],
 ['5', '33', 'F', 'other', '15213']]

# 建立DataFrame

In [8]:
sqlContext = SparkSession.builder.getOrCreate() # 获取或创建
print(sqlContext)

In [9]:
from pyspark.sql import Row
user_Rows = userRDD.map(lambda p:
    Row(
        userid=int(p[0]),
        age=int(p[1]),
        gender=p[2],
        occupation=p[3],
        zipcode=p[4]
    )
)
user_Rows.take(5)

[Row(age=24, gender='M', occupation='technician', userid=1, zipcode='85711'),
 Row(age=53, gender='F', occupation='other', userid=2, zipcode='94043'),
 Row(age=23, gender='M', occupation='writer', userid=3, zipcode='32067'),
 Row(age=24, gender='M', occupation='technician', userid=4, zipcode='43537'),
 Row(age=33, gender='F', occupation='other', userid=5, zipcode='15213')]

In [10]:
user_df = sqlContext.createDataFrame(user_Rows)
user_df.printSchema() # 输出结构元数据信息

root
 |-- age: long (nullable = true)
 |-- gender: string (nullable = true)
 |-- occupation: string (nullable = true)
 |-- userid: long (nullable = true)
 |-- zipcode: string (nullable = true)



In [11]:
print(type(user_df))

<class 'pyspark.sql.dataframe.DataFrame'>


In [12]:
user_df.show(5)

+---+------+----------+------+-------+
|age|gender|occupation|userid|zipcode|
+---+------+----------+------+-------+
| 24|     M|technician|     1|  85711|
| 53|     F|     other|     2|  94043|
| 23|     M|    writer|     3|  32067|
| 24|     M|technician|     4|  43537|
| 33|     F|     other|     5|  15213|
+---+------+----------+------+-------+
only showing top 5 rows



In [13]:
df = user_df.alias("df") # 取别名
df.show(5)

+---+------+----------+------+-------+
|age|gender|occupation|userid|zipcode|
+---+------+----------+------+-------+
| 24|     M|technician|     1|  85711|
| 53|     F|     other|     2|  94043|
| 23|     M|    writer|     3|  32067|
| 24|     M|technician|     4|  43537|
| 33|     F|     other|     5|  15213|
+---+------+----------+------+-------+
only showing top 5 rows



# 建立Spark SQL tempTable

In [14]:
user_df.registerTempTable("user_table")

In [15]:
sqlContext.sql("SELECT count(*) counts FROM user_table").show()

+------+
|counts|
+------+
|   943|
+------+



# 显示部分字段

In [16]:
userRDDnew = userRDD.map(lambda x : (x[2], x[3], x[0]))
userRDDnew.take(5)

[('M', 'technician', '1'),
 ('F', 'other', '2'),
 ('M', 'writer', '3'),
 ('M', 'technician', '4'),
 ('F', 'other', '5')]

In [17]:
user_df.select("userid", "occupation", "gender", "age").show(5)

+------+----------+------+---+
|userid|occupation|gender|age|
+------+----------+------+---+
|     1|technician|     M| 24|
|     2|     other|     F| 53|
|     3|    writer|     M| 23|
|     4|technician|     M| 24|
|     5|     other|     F| 33|
+------+----------+------+---+
only showing top 5 rows



In [18]:
user_df.select(user_df.userid, user_df.occupation, user_df.gender, user_df.age).show(5)

+------+----------+------+---+
|userid|occupation|gender|age|
+------+----------+------+---+
|     1|technician|     M| 24|
|     2|     other|     F| 53|
|     3|    writer|     M| 23|
|     4|technician|     M| 24|
|     5|     other|     F| 33|
+------+----------+------+---+
only showing top 5 rows



In [19]:
sqlContext.sql("SELECT userid, occupation, gender, age FROM user_table").show(5)

+------+----------+------+---+
|userid|occupation|gender|age|
+------+----------+------+---+
|     1|technician|     M| 24|
|     2|     other|     F| 53|
|     3|    writer|     M| 23|
|     4|technician|     M| 24|
|     5|     other|     F| 33|
+------+----------+------+---+
only showing top 5 rows



# 增加计算字段

In [20]:
userRDDnew = userRDD.map(lambda x : (x[0], x[3], x[2], x[1], 2020-int(x[1])))
userRDDnew.take(5)

[('1', 'technician', 'M', '24', 1996),
 ('2', 'other', 'F', '53', 1967),
 ('3', 'writer', 'M', '23', 1997),
 ('4', 'technician', 'M', '24', 1996),
 ('5', 'other', 'F', '33', 1987)]

In [21]:
df.select("userid", "occupation", "gender", "age", 2019-df.age).show(5)

+------+----------+------+---+------------+
|userid|occupation|gender|age|(2019 - age)|
+------+----------+------+---+------------+
|     1|technician|     M| 24|        1995|
|     2|     other|     F| 53|        1966|
|     3|    writer|     M| 23|        1996|
|     4|technician|     M| 24|        1995|
|     5|     other|     F| 33|        1986|
+------+----------+------+---+------------+
only showing top 5 rows



In [22]:
df.select("userid", "occupation", "gender", "age", (2020-df.age).alias("birthyear")).show(5)

+------+----------+------+---+---------+
|userid|occupation|gender|age|birthyear|
+------+----------+------+---+---------+
|     1|technician|     M| 24|     1996|
|     2|     other|     F| 53|     1967|
|     3|    writer|     M| 23|     1997|
|     4|technician|     M| 24|     1996|
|     5|     other|     F| 33|     1987|
+------+----------+------+---+---------+
only showing top 5 rows



In [23]:
sqlContext.sql("SELECT userid, occupation, gender, age, 2020-age birthyear FROM user_table").show(5)

+------+----------+------+---+---------+
|userid|occupation|gender|age|birthyear|
+------+----------+------+---+---------+
|     1|technician|     M| 24|     1996|
|     2|     other|     F| 53|     1967|
|     3|    writer|     M| 23|     1997|
|     4|technician|     M| 24|     1996|
|     5|     other|     F| 33|     1987|
+------+----------+------+---+---------+
only showing top 5 rows



# 筛选数据

In [24]:
userRDD.filter(lambda r : r[3] == 'technician' and r[2] == 'M' and r[1] == '24').take(6)

[['1', '24', 'M', 'technician', '85711'],
 ['4', '24', 'M', 'technician', '43537'],
 ['456', '24', 'M', 'technician', '31820'],
 ['717', '24', 'M', 'technician', '84105'],
 ['832', '24', 'M', 'technician', '77042'],
 ['889', '24', 'M', 'technician', '78704']]

In [25]:
user_df.filter("occupation='technician'").filter("gender='M'").filter("age=24").show(3)

+---+------+----------+------+-------+
|age|gender|occupation|userid|zipcode|
+---+------+----------+------+-------+
| 24|     M|technician|     1|  85711|
| 24|     M|technician|     4|  43537|
| 24|     M|technician|   456|  31820|
+---+------+----------+------+-------+
only showing top 3 rows



In [26]:
user_df.filter("occupation='technician' and gender='M' and age=24").show()

+---+------+----------+------+-------+
|age|gender|occupation|userid|zipcode|
+---+------+----------+------+-------+
| 24|     M|technician|     1|  85711|
| 24|     M|technician|     4|  43537|
| 24|     M|technician|   456|  31820|
| 24|     M|technician|   717|  84105|
| 24|     M|technician|   832|  77042|
| 24|     M|technician|   889|  78704|
+---+------+----------+------+-------+



In [27]:
df.filter((df.occupation=='technician') & (df.gender=='M') & (df.age==24)).show()

+---+------+----------+------+-------+
|age|gender|occupation|userid|zipcode|
+---+------+----------+------+-------+
| 24|     M|technician|     1|  85711|
| 24|     M|technician|     4|  43537|
| 24|     M|technician|   456|  31820|
| 24|     M|technician|   717|  84105|
| 24|     M|technician|   832|  77042|
| 24|     M|technician|   889|  78704|
+---+------+----------+------+-------+



In [28]:
df.filter((df['occupation']=='technician') & (df['gender']=='M') & (df['age']==24)).show()

+---+------+----------+------+-------+
|age|gender|occupation|userid|zipcode|
+---+------+----------+------+-------+
| 24|     M|technician|     1|  85711|
| 24|     M|technician|     4|  43537|
| 24|     M|technician|   456|  31820|
| 24|     M|technician|   717|  84105|
| 24|     M|technician|   832|  77042|
| 24|     M|technician|   889|  78704|
+---+------+----------+------+-------+



In [29]:
sqlContext.sql("SELECT * FROM user_table where occupation='technician' and gender='M' and age=24").show(5)

+---+------+----------+------+-------+
|age|gender|occupation|userid|zipcode|
+---+------+----------+------+-------+
| 24|     M|technician|     1|  85711|
| 24|     M|technician|     4|  43537|
| 24|     M|technician|   456|  31820|
| 24|     M|technician|   717|  84105|
| 24|     M|technician|   832|  77042|
+---+------+----------+------+-------+
only showing top 5 rows



# 单个字段排序

In [30]:
# ascending=0 降序
# ascending=1 升序
df.select("userid", "occupation", "gender", "age").orderBy("age", ascending=0).show(5)

+------+-------------+------+---+
|userid|   occupation|gender|age|
+------+-------------+------+---+
|   481|      retired|     M| 73|
|   767|     engineer|     M| 70|
|   860|      retired|     F| 70|
|   803|administrator|     M| 70|
|   559|    executive|     M| 69|
+------+-------------+------+---+
only showing top 5 rows



In [31]:
df.select("userid", "occupation", "gender", "age").orderBy(df.age).show(5)

+------+----------+------+---+
|userid|occupation|gender|age|
+------+----------+------+---+
|    30|   student|     M|  7|
|   471|   student|     M| 10|
|   289|      none|     M| 11|
|   142|     other|     M| 13|
|   609|   student|     F| 13|
+------+----------+------+---+
only showing top 5 rows



In [32]:
df.select("userid", "occupation", "gender", "age").orderBy(df.age.desc()).show(5)

+------+-------------+------+---+
|userid|   occupation|gender|age|
+------+-------------+------+---+
|   481|      retired|     M| 73|
|   767|     engineer|     M| 70|
|   860|      retired|     F| 70|
|   803|administrator|     M| 70|
|   559|    executive|     M| 69|
+------+-------------+------+---+
only showing top 5 rows



In [33]:
# 按照年龄升序
sqlContext.sql("SELECT userid, occupation, gender, age FROM user_table ORDER BY age").show(5)

+------+----------+------+---+
|userid|occupation|gender|age|
+------+----------+------+---+
|    30|   student|     M|  7|
|   471|   student|     M| 10|
|   289|      none|     M| 11|
|   880|   student|     M| 13|
|   628|      none|     M| 13|
+------+----------+------+---+
only showing top 5 rows



In [34]:
# 按照年龄降序
sqlContext.sql("SELECT userid, occupation, gender, age FROM user_table ORDER BY age DESC").show(5)

+------+-------------+------+---+
|userid|   occupation|gender|age|
+------+-------------+------+---+
|   481|      retired|     M| 73|
|   860|      retired|     F| 70|
|   767|     engineer|     M| 70|
|   803|administrator|     M| 70|
|   559|    executive|     M| 69|
+------+-------------+------+---+
only showing top 5 rows



# 多字段排序

In [35]:
userRDD.takeOrdered(5, key = lambda x : (-int(x[1]), x[2]))

[['481', '73', 'M', 'retired', '37771'],
 ['860', '70', 'F', 'retired', '48322'],
 ['767', '70', 'M', 'engineer', '00000'],
 ['803', '70', 'M', 'administrator', '78212'],
 ['559', '69', 'M', 'executive', '10022']]

In [36]:
sqlContext.sql("""
SELECT userid, age, gender, occupation, zipcode
FROM user_table
ORDER BY age DESC, gender """).show(5)

+------+---+------+-------------+-------+
|userid|age|gender|   occupation|zipcode|
+------+---+------+-------------+-------+
|   481| 73|     M|      retired|  37771|
|   860| 70|     F|      retired|  48322|
|   803| 70|     M|administrator|  78212|
|   767| 70|     M|     engineer|  00000|
|   559| 69|     M|    executive|  10022|
+------+---+------+-------------+-------+
only showing top 5 rows



In [37]:
df.orderBy(["age", "gender"], ascending=[0, 1]).show(5) # 年龄降序，性别升序

+---+------+-------------+------+-------+
|age|gender|   occupation|userid|zipcode|
+---+------+-------------+------+-------+
| 73|     M|      retired|   481|  37771|
| 70|     F|      retired|   860|  48322|
| 70|     M|administrator|   803|  78212|
| 70|     M|     engineer|   767|  00000|
| 69|     M|    executive|   559|  10022|
+---+------+-------------+------+-------+
only showing top 5 rows



In [38]:
df.orderBy(df.age.desc(), df.gender).show(5)

+---+------+-------------+------+-------+
|age|gender|   occupation|userid|zipcode|
+---+------+-------------+------+-------+
| 73|     M|      retired|   481|  37771|
| 70|     F|      retired|   860|  48322|
| 70|     M|     engineer|   767|  00000|
| 70|     M|administrator|   803|  78212|
| 69|     M|    executive|   559|  10022|
+---+------+-------------+------+-------+
only showing top 5 rows



# 显示不重复数据

In [39]:
userRDD.map(lambda x : x[2]).distinct().collect()

['M', 'F']

In [40]:
userRDD.map(lambda x : (x[1], x[2])).distinct().take(20) # 多个列共同确定是否唯一

[('23', 'M'),
 ('42', 'M'),
 ('36', 'M'),
 ('39', 'F'),
 ('28', 'F'),
 ('47', 'M'),
 ('49', 'F'),
 ('30', 'M'),
 ('35', 'F'),
 ('42', 'F'),
 ('25', 'M'),
 ('30', 'F'),
 ('39', 'M'),
 ('49', 'M'),
 ('32', 'M'),
 ('41', 'M'),
 ('7', 'M'),
 ('38', 'F'),
 ('38', 'M'),
 ('27', 'F')]

In [41]:
sqlContext.sql("SELECT distinct gender FROM user_table").show()

+------+
|gender|
+------+
|     F|
|     M|
+------+



In [42]:
sqlContext.sql("SELECT distinct age, gender FROM user_table").show()

+---+------+
|age|gender|
+---+------+
| 39|     F|
| 48|     M|
| 26|     M|
| 28|     M|
| 54|     M|
| 60|     M|
| 50|     M|
| 53|     F|
| 30|     M|
| 48|     F|
| 47|     M|
| 46|     M|
| 56|     M|
| 32|     M|
| 31|     M|
| 53|     M|
| 20|     F|
| 29|     F|
| 21|     F|
| 42|     M|
+---+------+
only showing top 20 rows



In [43]:
user_df.select("gender").distinct().show()

+------+
|gender|
+------+
|     F|
|     M|
+------+



In [44]:
user_df.select("gender", "age").distinct().show()

+------+---+
|gender|age|
+------+---+
|     F| 29|
|     F| 47|
|     M| 56|
|     M| 53|
|     F| 28|
|     M| 33|
|     F| 49|
|     F| 40|
|     F| 13|
|     F| 57|
|     F| 23|
|     F| 20|
|     F| 59|
|     M| 60|
|     F| 31|
|     F| 33|
|     M| 15|
|     F| 32|
|     M| 17|
|     M| 32|
+------+---+
only showing top 20 rows



In [45]:
user_df.select("gender")\
        .groupby("gender")\
        .count().show()

+------+-----+
|gender|count|
+------+-----+
|     F|  273|
|     M|  670|
+------+-----+



In [46]:
user_df.select("gender", "occupation").\
                groupby("gender", "occupation").\
                count().\
                orderBy("gender", "occupation").\
                show(100)

+------+-------------+-----+
|gender|   occupation|count|
+------+-------------+-----+
|     F|administrator|   36|
|     F|       artist|   13|
|     F|     educator|   26|
|     F|     engineer|    2|
|     F|entertainment|    2|
|     F|    executive|    3|
|     F|   healthcare|   11|
|     F|    homemaker|    6|
|     F|       lawyer|    2|
|     F|    librarian|   29|
|     F|    marketing|   10|
|     F|         none|    4|
|     F|        other|   36|
|     F|   programmer|    6|
|     F|      retired|    1|
|     F|     salesman|    3|
|     F|    scientist|    3|
|     F|      student|   60|
|     F|   technician|    1|
|     F|       writer|   19|
|     M|administrator|   43|
|     M|       artist|   15|
|     M|       doctor|    7|
|     M|     educator|   69|
|     M|     engineer|   65|
|     M|entertainment|   16|
|     M|    executive|   29|
|     M|   healthcare|    5|
|     M|    homemaker|    1|
|     M|       lawyer|   10|
|     M|    librarian|   22|
|     M|    ma

In [47]:
user_df.stat.crosstab("occupation", "gender").show(30) # 交叉表

+-----------------+---+---+
|occupation_gender|  F|  M|
+-----------------+---+---+
|        scientist|  3| 28|
|          student| 60|136|
|           writer| 19| 26|
|         salesman|  3|  9|
|          retired|  1| 13|
|    administrator| 36| 43|
|       programmer|  6| 60|
|           doctor|  0|  7|
|        homemaker|  6|  1|
|        executive|  3| 29|
|         engineer|  2| 65|
|    entertainment|  2| 16|
|        marketing| 10| 16|
|       technician|  1| 26|
|           artist| 13| 15|
|        librarian| 29| 22|
|           lawyer|  2| 10|
|         educator| 26| 69|
|       healthcare| 11|  5|
|             none|  4|  5|
|            other| 36| 69|
+-----------------+---+---+



In [48]:
sqlContext.sql("""
SELECT gender, occupation, count(*) counts
FROM user_table
GROUP BY gender, occupation
""").show(100)

+------+-------------+------+
|gender|   occupation|counts|
+------+-------------+------+
|     M|    executive|    29|
|     M|     educator|    69|
|     F|         none|     4|
|     F|entertainment|     2|
|     F|      retired|     1|
|     F|       artist|    13|
|     F|    librarian|    29|
|     F|     engineer|     2|
|     F|   healthcare|    11|
|     F|administrator|    36|
|     M|        other|    69|
|     M|    homemaker|     1|
|     F|       lawyer|     2|
|     M|   programmer|    60|
|     M|     salesman|     9|
|     M|         none|     5|
|     M|entertainment|    16|
|     M|    marketing|    16|
|     M|   technician|    26|
|     M|administrator|    43|
|     F|    marketing|    10|
|     F|   programmer|     6|
|     F|   technician|     1|
|     F|    executive|     3|
|     M|    scientist|    28|
|     F|     educator|    26|
|     M|      retired|    13|
|     M|   healthcare|     5|
|     M|       writer|    26|
|     M|       lawyer|    10|
|     M|  

In [49]:
user_df.select("gender").groupby("gender").count().show()

+------+-----+
|gender|count|
+------+-----+
|     F|  273|
|     M|  670|
+------+-----+



In [50]:
user_df.select("gender", "occupation").groupby("gender", "occupation").count().orderBy("gender", "occupation").show()

+------+-------------+-----+
|gender|   occupation|count|
+------+-------------+-----+
|     F|administrator|   36|
|     F|       artist|   13|
|     F|     educator|   26|
|     F|     engineer|    2|
|     F|entertainment|    2|
|     F|    executive|    3|
|     F|   healthcare|   11|
|     F|    homemaker|    6|
|     F|       lawyer|    2|
|     F|    librarian|   29|
|     F|    marketing|   10|
|     F|         none|    4|
|     F|        other|   36|
|     F|   programmer|    6|
|     F|      retired|    1|
|     F|     salesman|    3|
|     F|    scientist|    3|
|     F|      student|   60|
|     F|   technician|    1|
|     F|       writer|   19|
+------+-------------+-----+
only showing top 20 rows



In [51]:
user_df.describe().show()

+-------+-----------------+------+-------------+-----------------+------------------+
|summary|              age|gender|   occupation|           userid|           zipcode|
+-------+-----------------+------+-------------+-----------------+------------------+
|  count|              943|   943|          943|              943|               943|
|   mean|34.05196182396607|  null|         null|            472.0| 50868.78810810811|
| stddev|12.19273973305903|  null|         null|272.3649512449549|30891.373254138158|
|    min|                7|     F|administrator|                1|             00000|
|    max|               73|     M|       writer|              943|             Y1A6B|
+-------+-----------------+------+-------------+-----------------+------------------+



# 预处理邮编数据集

In [52]:
rawDataWithHeader = sc.textFile(Path + "free-zipcode-database-Primary.csv")
rawDataWithHeader.take(2)

['"Zipcode","ZipCodeType","City","State","LocationType","Lat","Long","Location","Decommisioned","TaxReturnsFiled","EstimatedPopulation","TotalWages"',
 '"00705","STANDARD","AIBONITO","PR","PRIMARY",18.14,-66.26,"NA-US-PR-AIBONITO","false",,,']

In [53]:
# 把字段过滤掉
header = rawDataWithHeader.first()
rawData = rawDataWithHeader.filter(lambda x : x != header)
rawData.first()

'"00705","STANDARD","AIBONITO","PR","PRIMARY",18.14,-66.26,"NA-US-PR-AIBONITO","false",,,'

In [54]:
rData = rawData.map(lambda x : x.replace("\"", ""))
rData.first()

'00705,STANDARD,AIBONITO,PR,PRIMARY,18.14,-66.26,NA-US-PR-AIBONITO,false,,,'

In [55]:
ZipRDD = rData.map(lambda x : x.split(","))
ZipRDD.first()

['00705',
 'STANDARD',
 'AIBONITO',
 'PR',
 'PRIMARY',
 '18.14',
 '-66.26',
 'NA-US-PR-AIBONITO',
 'false',
 '',
 '',
 '']

## 建立DataFrame与临时表zipcode_table

In [56]:
from pyspark.sql import Row
zipcode_data = ZipRDD.map(lambda p :
    Row(
        zipcode=int(p[0]),
        zipCodeType=p[1],
        city=p[2],
        state=p[3]
    )
)
zipcode_data.take(5)

[Row(city='AIBONITO', state='PR', zipCodeType='STANDARD', zipcode=705),
 Row(city='ANASCO', state='PR', zipCodeType='STANDARD', zipcode=610),
 Row(city='ANGELES', state='PR', zipCodeType='PO BOX', zipcode=611),
 Row(city='ARECIBO', state='PR', zipCodeType='STANDARD', zipcode=612),
 Row(city='ADJUNTAS', state='PR', zipCodeType='STANDARD', zipcode=601)]

In [57]:
zipcode_df = sqlContext.createDataFrame(zipcode_data)
zipcode_df.printSchema()

root
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- zipCodeType: string (nullable = true)
 |-- zipcode: long (nullable = true)



In [58]:
zipcode_df.registerTempTable("zipcode_table")
zipcode_df.show(10)

+---------+-----+-----------+-------+
|     city|state|zipCodeType|zipcode|
+---------+-----+-----------+-------+
| AIBONITO|   PR|   STANDARD|    705|
|   ANASCO|   PR|   STANDARD|    610|
|  ANGELES|   PR|     PO BOX|    611|
|  ARECIBO|   PR|   STANDARD|    612|
| ADJUNTAS|   PR|   STANDARD|    601|
| CASTANER|   PR|     PO BOX|    631|
|   AGUADA|   PR|   STANDARD|    602|
|AGUADILLA|   PR|   STANDARD|    603|
|AGUADILLA|   PR|     PO BOX|    604|
|AGUADILLA|   PR|     PO BOX|    605|
+---------+-----+-----------+-------+
only showing top 10 rows



In [59]:
sqlContext.sql("""
SELECT z.*
FROM zipcode_table z
""").show(10)

+---------+-----+-----------+-------+
|     city|state|zipCodeType|zipcode|
+---------+-----+-----------+-------+
| AIBONITO|   PR|   STANDARD|    705|
|   ANASCO|   PR|   STANDARD|    610|
|  ANGELES|   PR|     PO BOX|    611|
|  ARECIBO|   PR|   STANDARD|    612|
| ADJUNTAS|   PR|   STANDARD|    601|
| CASTANER|   PR|     PO BOX|    631|
|   AGUADA|   PR|   STANDARD|    602|
|AGUADILLA|   PR|   STANDARD|    603|
|AGUADILLA|   PR|     PO BOX|    604|
|AGUADILLA|   PR|     PO BOX|    605|
+---------+-----+-----------+-------+
only showing top 10 rows



In [60]:
sqlContext.sql("select * from user_table").show(10)

+---+------+-------------+------+-------+
|age|gender|   occupation|userid|zipcode|
+---+------+-------------+------+-------+
| 24|     M|   technician|     1|  85711|
| 53|     F|        other|     2|  94043|
| 23|     M|       writer|     3|  32067|
| 24|     M|   technician|     4|  43537|
| 33|     F|        other|     5|  15213|
| 42|     M|    executive|     6|  98101|
| 57|     M|administrator|     7|  91344|
| 36|     M|administrator|     8|  05201|
| 29|     M|      student|     9|  01002|
| 53|     M|       lawyer|    10|  90703|
+---+------+-------------+------+-------+
only showing top 10 rows



In [61]:
sqlContext.sql("""
SELECT u.*, z.city, z.state
FROM user_table u
LEFT JOIN zipcode_table z ON u.zipcode = z.zipcode
WHERE z.state='NY'
""").show(10)

+---+------+-------------+------+-------+----------------+-----+
|age|gender|   occupation|userid|zipcode|            city|state|
+---+------+-------------+------+-------+----------------+-----+
| 22|     F|   healthcare|   405|  10019|        NEW YORK|   NY|
| 29|     M|        other|   478|  10019|        NEW YORK|   NY|
| 22|     M|      student|   327|  11101|LONG ISLAND CITY|   NY|
| 48|     M|     educator|   656|  10314|   STATEN ISLAND|   NY|
| 27|     F|       writer|   617|  11201|        BROOKLYN|   NY|
| 35|     F|        other|   760|  14211|         BUFFALO|   NY|
| 32|     F|        other|   155|  11217|        BROOKLYN|   NY|
| 30|     F|       writer|   557|  11217|        BROOKLYN|   NY|
| 27|     M|    marketing|   806|  11217|        BROOKLYN|   NY|
| 23|     M|administrator|   509|  10011|        NEW YORK|   NY|
+---+------+-------------+------+-------+----------------+-----+
only showing top 10 rows

